In [76]:
import os

In [77]:
%pwd

'/Users/ujjwaltiwari/Desktop/projects/kidney-disease-classifier-ct-scan'

In [3]:
os.chdir('../')

In [94]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/UjjTiw/kidney-disease-classifier-ct-scan.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'UjjTiw'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'aed5cdc8e6331b6bacc6a684372ea07f6d913399'

In [95]:
import tensorflow as tf

In [96]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int


In [97]:
from DiseaseClassifier.constants import  *
from DiseaseClassifier.utils.common import read_yaml, create_directories, save_json

In [98]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= 'artifacts/training/model.h5',
            training_data= 'artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone',
            mlflow_uri= 'https://dagshub.com/UjjTiw/kidney-disease-classifier-ct-scan.mlflow',
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )
        return eval_config
    

In [99]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.tensorflow
from urllib.parse import urlparse

In [100]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            # if tracking_url_type_store != "file":

            #     # Register the model
            #     # There are other ways to use the Model Registry, which depends on the use case,
            #     # please refer to the doc for more information:
            #     # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            #     mlflow.tensorflow.log_model(self.model, "model", registered_model_name="EfficientNetB7")
            # else:
            #     mlflow.tensorflow.log_model(self.model, "model")

In [102]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation  = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-04-16 12:04:58,394: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-16 12:04:58,400: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-16 12:04:58,401: INFO: common: Created directory at : artifacts]
Found 3732 images belonging to 4 classes.
117/117 [==============================] - 358s 3s/step - loss: 118.1015 - accuracy: 0.4767
[2024-04-16 12:11:03,172: INFO: common: json files saved at: scores.json]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/ujjwaltiwari/anaconda3/envs/kidney_project/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/x5/z5cwd1cx4wzdf6bwhq_b41x40000gn/T/ipykernel_38984/1316878791.py", line 9, in <module>
    raise e
  File "/var/folders/x5/z5cwd1cx4wzdf6bwhq_b41x40000gn/T/ipykernel_38984/1316878791.py", line 6, in <module>
    evaluation.log_into_mlflow()
  File "/var/folders/x5/z5cwd1cx4wzdf6bwhq_b41x40000gn/T/ipykernel_38984/1741163121.py", line 51, in log_into_mlflow
    with mlflow.start_run():
  File "/Users/ujjwaltiwari/anaconda3/envs/kidney_project/lib/python3.9/site-packages/mlflow/tracking/fluent.py", line 350, in start_run
    error_code=INVALID_PARAMETER_VALUE,
  File "/Users/ujjwaltiwari/anaconda3/envs/kidney_project/lib/python3.9/site-packages/mlflow/tracking/client.py", line 275, in create_run
    --
  File "/Users/ujjwaltiwari/anaconda

In [ ]:
hmm = tf.keras.models.load_model('artifacts/training/model.h5')
type(hmm)

keras.src.engine.functional.Functional